# 使用 Collection Schema 的 Chatbot

## 回顾

我们扩展了我们的 chatbot,将语义记忆保存到单一的 [用户 profile](https://langchain-ai.github.io/langgraph/concepts/memory/#profile) 中。

我们还介绍了一个库,[Trustcall](https://github.com/hinthornw/trustcall),用于使用新信息更新这个 schema。

## 目标

有时我们希望将记忆保存到 [collection](https://docs.google.com/presentation/d/181mvjlgsnxudQI6S3ritg9sooNyu4AcLLFH1UK0kIuk/edit#slide=id.g30eb3c8cf10_0_200) 而不是单一的 profile。

在这里,我们将更新我们的 chatbot 来[将记忆保存到 collection](https://langchain-ai.github.io/langgraph/concepts/memory/#collection)。

我们还将展示如何使用 [Trustcall](https://github.com/hinthornw/trustcall) 来更新这个 collection。

In [ ]:
%%capture --no-stderr
%pip install -U langchain_openai langgraph trustcall langchain_core

In [ ]:
import os, getpass

def _set_env(var: str):
    """
    辅助函数 - 设置环境变量
    
    Python 知识点:
    - os.environ: Python 的环境变量字典
    - getpass.getpass(): 安全地提示用户输入密码
    
    工作流程:
        1. 检查环境变量是否已设置
        2. 如果未设置,提示用户输入
        3. 设置到当前进程的环境变量中
    """
    # 检查环境变量是否已在 OS 环境中设置
    env_value = os.environ.get(var)
    if not env_value:
        # 如果未设置,提示用户输入
        env_value = getpass.getpass(f"{var}: ")
    
    # 为当前进程设置环境变量
    os.environ[var] = env_value

# ================== LangSmith 配置 ==================
# LangSmith: LangChain 的追踪和调试平台

# 设置 LangSmith API 密钥
_set_env("LANGSMITH_API_KEY")

# 启用 LangSmith 追踪功能
os.environ["LANGSMITH_TRACING"] = "true"

# 设置 LangSmith 项目名称
os.environ["LANGSMITH_PROJECT"] = "langchain-academy"

## 定义 collection schema

与其将用户信息存储在固定的 profile 结构中,我们将创建一个灵活的 collection schema 来存储关于用户互动的记忆。

每个记忆将作为单独的条目存储,只有一个 `content` 字段用于存储我们想要记住的主要信息。

这种方法允许我们构建一个开放式的记忆 collection,可以随着我们对用户了解的增加而增长和变化。

我们可以将 collection schema 定义为 [Pydantic](https://docs.pydantic.dev/latest/) 对象。

In [ ]:
from pydantic import BaseModel, Field

# ================== 定义 Collection Schema ==================

# Collection vs Profile:
# - Profile: 单一的、结构化的用户档案,包含固定字段(name, location, interests)
# - Collection: 多个独立的记忆条目,每个条目是一个独立的 Memory 对象
# 
# Collection 的优势:
# - 灵活性: 可以添加任意数量的记忆,无需预定义结构
# - 独立性: 每个记忆可以独立更新或删除
# - 扩展性: 适合存储开放式的、不断增长的信息

class Memory(BaseModel):
    """
    单个记忆条目 - Collection 中的基本单元
    
    Pydantic 知识点:
    - BaseModel: Pydantic 的基类,提供数据验证和序列化
    - Field: 定义字段的元数据,如描述、验证规则等
    
    为什么只有一个字段:
    - 简单性: 每个记忆只包含核心内容
    - 灵活性: content 可以包含各种类型的信息
    - 可扩展: 未来可以添加更多字段(timestamp, importance, etc.)
    """
    content: str = Field(
        description="The main content of the memory. For example: User expressed interest in learning about French."
    )

class MemoryCollection(BaseModel):
    """
    记忆集合 - 包含多个 Memory 的列表
    
    用途:
    - 用于 with_structured_output 一次性提取多个记忆
    - 不直接保存到 Store,而是将每个 Memory 单独保存
    """
    memories: list[Memory] = Field(
        description="A list of memories about the user."
    )

In [3]:
_set_env("OPENAI_API_KEY")

我们可以使用 LangChain 的 [chat model](https://python.langchain.com/docs/concepts/chat_models/) 接口的 [`with_structured_output`](https://python.langchain.com/docs/concepts/structured_outputs/#recommended-usage) 方法来强制执行结构化输出。

In [ ]:
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI

# ================== 使用 with_structured_output 提取记忆 ==================

# 初始化 model
model = ChatOpenAI(model="gpt-4o", temperature=0)

# 将 MemoryCollection schema 绑定到 model
# LangChain 知识点:
# - with_structured_output: 强制 LLM 输出符合指定 schema 的数据
# - 返回的是 MemoryCollection 实例,包含 memories 列表
model_with_structure = model.with_structured_output(MemoryCollection)

# ================== 调用 Model 提取记忆 ==================
# 工作流程:
# 1. LLM 分析输入消息
# 2. 识别多个独立的信息点
# 3. 为每个信息点创建一个 Memory 对象
# 4. 返回 MemoryCollection 包含所有 Memory

# 调用 model 从用户消息中提取记忆
# 输入: "My name is Lance. I like to bike."
# 预期输出: MemoryCollection 包含两个 Memory:
#   1. Memory(content="User's name is Lance.")
#   2. Memory(content='Lance likes to bike.')
memory_collection = model_with_structure.invoke([HumanMessage("My name is Lance. I like to bike.")])

# 显示提取的记忆列表
memory_collection.memories

我们可以使用 `model_dump()` 将 Pydantic model 实例序列化为 Python 字典。

In [ ]:
# ================== 序列化 Pydantic Model ==================

# Python 知识点:
# - model_dump(): Pydantic 方法,将 model 实例转换为 Python 字典
# - 这是保存到 Store 前的必要步骤,因为 Store 只接受字典

# 将第一个 Memory 转换为字典
# 输入: Memory(content="User's name is Lance.")
# 输出: {'content': "User's name is Lance."}
memory_collection.memories[0].model_dump()

将每个记忆的字典表示保存到 store。

In [ ]:
import uuid
from langgraph.store.memory import InMemoryStore

# ================== 初始化 Store ==================

# 初始化内存 store
in_memory_store = InMemoryStore()

# ================== 定义 Namespace ==================
# Collection 存储策略:
# - Namespace: (user_id, "memories") 
# - 注意: 使用 "memories" 而不是 "memory",表示这是多个记忆的集合
# - 每个用户有一个独立的 memories namespace

user_id = "1"
namespace_for_memory = (user_id, "memories")

# ================== 保存记忆到 Store ==================
# Collection 保存模式:
# - 每个 Memory 作为独立的条目保存
# - Key: 使用 UUID 生成唯一标识符
# - Value: Memory 的字典表示
# 
# 这与 Profile 不同:
# - Profile: 一个 key ("user_memory") 保存整个 profile
# - Collection: 多个 key (UUID) 每个保存一个 Memory

# Python 知识点:
# - uuid.uuid4(): 生成随机的 UUID (Universally Unique Identifier)
# - str(uuid): 将 UUID 对象转换为字符串

# 保存第一个记忆
key = str(uuid.uuid4())  # 生成唯一 key,例如: "e1c4e5ab-ab0f-4cbb-822d-f29240a983af"
value = memory_collection.memories[0].model_dump()  # {'content': "User's name is Lance."}
in_memory_store.put(namespace_for_memory, key, value)

# 保存第二个记忆
key = str(uuid.uuid4())  # 生成另一个唯一 key
value = memory_collection.memories[1].model_dump()  # {'content': 'Lance likes to bike.'}
in_memory_store.put(namespace_for_memory, key, value)

# Collection 保存的优势:
# - 每个记忆可以独立更新或删除
# - 可以轻松添加新记忆而不影响现有记忆
# - 支持按需检索特定记忆

在 store 中搜索记忆。

In [ ]:
# ================== 搜索 Collection 中的记忆 ==================

# LangGraph Store 知识点:
# - search(namespace): 检索指定 namespace 下的所有记忆
# - 返回 Item 对象列表,每个包含 key, value, namespace, created_at, updated_at

# 搜索并打印所有记忆
# 输出格式:
# - value: 记忆内容 {'content': '...'}
# - key: UUID 标识符
# - namespace: ['1', 'memories']
# - created_at/updated_at: 时间戳
for m in in_memory_store.search(namespace_for_memory):
    print(m.dict())

# Collection 搜索的特点:
# - 返回多个独立的 Item 对象
# - 每个 Item 有唯一的 key
# - 可以按时间排序或过滤

## 更新 collection schema

我们在上一课中讨论了更新 profile schema 的挑战。

同样的问题也适用于 collections!

我们希望能够用新记忆更新 collection,以及更新 collection 中的现有记忆。

现在我们将展示 [Trustcall](https://github.com/hinthornw/trustcall) 也可以用于更新 collection。

这使得既可以添加新记忆,也可以[更新 collection 中的现有记忆](https://github.com/hinthornw/trustcall?tab=readme-ov-file#simultanous-updates--insertions)。

让我们使用 Trustcall 定义一个新的 extractor。

和之前一样,我们为每个记忆提供 schema,即 `Memory`。

但是,我们可以提供 `enable_inserts=True` 来允许 extractor 向 collection 插入新记忆。

In [ ]:
from trustcall import create_extractor

# ================== 创建 Trustcall Extractor for Collection ==================

# Trustcall 知识点:
# - create_extractor: 创建可以提取和更新结构化数据的 extractor
# - 与 Profile 不同,Collection 需要特殊配置

# 创建 extractor
trustcall_extractor = create_extractor(
    model,
    tools=[Memory],  # 注意: 传入 Memory 而不是 MemoryCollection
    tool_choice="Memory",  # 强制使用 Memory tool
    
    # ========== 关键参数: enable_inserts ==========
    # enable_inserts=True: 允许 extractor 插入新记忆到 collection
    # 
    # 工作模式:
    # - 如果 enable_inserts=False: 只能更新现有记忆
    # - 如果 enable_inserts=True: 可以同时更新现有记忆和插入新记忆
    # 
    # Collection 的核心特性:
    # - 并行更新和插入: Trustcall 可以同时进行多个操作
    # - 自动识别: LLM 决定是更新现有记忆还是创建新记忆
    enable_inserts=True,
)

# 与 Profile Extractor 的区别:
# 1. Profile: 不需要 enable_inserts,因为总是更新同一个对象
# 2. Collection: 需要 enable_inserts,因为要支持添加新记忆

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# ================== 准备提取指令 ==================

# 系统指令 - 告诉 LLM 从对话中提取记忆
instruction = """Extract memories from the following conversation:"""

# ================== 准备对话历史 ==================

# 对话历史包含三条消息
conversation = [
    HumanMessage(content="Hi, I'm Lance."),  # 包含名字信息
    AIMessage(content="Nice to meet you, Lance."),  # AI 响应
    HumanMessage(content="This morning I had a nice bike ride in San Francisco.")  # 包含活动信息
]

# ================== 调用 Trustcall Extractor ==================

# Trustcall 工作流程:
# 1. 接收 SystemMessage (指令) + 对话历史
# 2. 分析对话,识别需要记住的信息
# 3. 使用 tool calling 生成 Memory tool calls
# 4. 解析 tool calls 为 Memory 实例
# 5. 返回结果字典

# 调用 extractor
# 输入格式: {"messages": [SystemMessage, ...对话消息]}
result = trustcall_extractor.invoke({
    "messages": [SystemMessage(content=instruction)] + conversation
})

# 预期行为:
# - LLM 会识别出信息: "Lance had a nice bike ride in San Francisco this morning."
# - 合并多个信息点为一个连贯的记忆
# - 返回包含 messages, responses, response_metadata 的字典

In [ ]:
# ================== 查看 Tool Calls (Messages) ==================

# Trustcall 返回结果的第一部分: messages
# - 包含 AIMessage 和其中的 tool calls
# - Tool calls 是 LLM 生成的结构化函数调用

# 打印所有 messages
# 输出格式:
# - Tool Calls: Memory (call_id)
# - Args: content="记忆内容"
for m in result["messages"]:
    m.pretty_print()

# LangChain 知识点:
# - Tool calling: LLM 生成的函数调用,包含 tool 名称、call_id 和参数
# - AIMessage: 包含 tool_calls 字段,是一个 tool call 列表

In [ ]:
# ================== 查看 Responses (解析后的记忆) ==================

# Trustcall 返回结果的第二部分: responses
# - 包含解析后的 Memory 实例列表
# - 每个 Memory 实例符合我们定义的 schema

# 打印所有 responses
# 输出: Memory(content='Lance had a nice bike ride in San Francisco this morning.')
for m in result["responses"]: 
    print(m)

# Trustcall 知识点:
# - responses: tool calls 的解析结果
# - 自动验证和转换为 Pydantic model 实例
# - 确保数据符合 schema 定义

In [ ]:
# ================== 查看 Metadata (Tool Call 元数据) ==================

# Trustcall 返回结果的第三部分: response_metadata
# - 包含每个 response 的元数据
# - 最重要的是 'id' 字段,对应 tool call 的 call_id

# 打印所有 metadata
# 输出: {'id': 'call_Pj4kctFlpg9TgcMBfMH33N30'}
for m in result["response_metadata"]: 
    print(m)

# Metadata 的用途:
# - id: 将 response 与对应的 tool call 关联
# - json_doc_id: (可选) 如果是更新操作,标识被更新的文档
# - 用于追踪和调试

In [ ]:
# ================== 准备更新对话 ==================

# 扩展对话 - 添加新的交互
updated_conversation = [
    AIMessage(content="That's great, did you do after?"),  # AI 询问后续
    HumanMessage(content="I went to Tartine and ate a croissant."),  # 新信息: 去了面包店
    AIMessage(content="What else is on your mind?"),  # AI 询问其他
    HumanMessage(content="I was thinking about my Japan, and going back this winter!"),  # 新信息: 日本计划
]

# ================== 更新系统指令 ==================

# 关键变化: "Update existing memories and create new ones"
# - 告诉 Trustcall 这是更新操作,不是从头创建
# - Trustcall 会决定:
#   1. 哪些信息应该更新现有记忆
#   2. 哪些信息应该创建新记忆
system_msg = """Update existing memories and create new ones based on the following conversation:"""

# ================== 准备 existing 参数 ==================

# Trustcall Collection 更新的核心: existing 参数
# - 格式: List[Tuple[id, tool_name, value]]
# - 每个元组包含三个元素:
#   1. id: 记忆的标识符 (字符串)
#   2. tool_name: tool 名称 ("Memory")
#   3. value: 记忆的字典表示

# Python 知识点:
# - enumerate(list): 返回 (index, item) 元组
# - str(i): 将索引转换为字符串作为 id
# - memory.model_dump(): 将 Memory 实例转换为字典

tool_name = "Memory"

# 创建 existing_memories 列表
# 格式: [('0', 'Memory', {'content': '...'})]
existing_memories = (
    [(str(i), tool_name, memory.model_dump()) 
     for i, memory in enumerate(result["responses"])]  # 遍历之前提取的记忆
    if result["responses"]  # 如果有记忆
    else None  # 否则为 None
)

# 输出示例:
# [('0', 'Memory', {'content': 'Lance had a nice bike ride in San Francisco this morning.'})]
existing_memories

# Collection 更新的关键点:
# - id (这里是 '0'): 用于标识要更新的记忆
# - Trustcall 会使用这个 id 来决定是更新还是插入

In [ ]:
# ================== 调用 Trustcall 进行更新和插入 ==================

# Trustcall Collection 更新机制:
# 1. 分析 updated_conversation 中的新信息
# 2. 比对 existing_memories 中的现有记忆
# 3. 决定哪些记忆需要更新,哪些需要插入
# 4. 生成相应的 tool calls
# 5. 返回更新后的结果

# 调用 extractor
result = trustcall_extractor.invoke({
    "messages": updated_conversation,  # 新的对话内容
    "existing": existing_memories  # 现有记忆列表
})

# Trustcall 的智能决策:
# - 如果新信息与现有记忆相关: 更新现有记忆 (标记 json_doc_id)
# - 如果新信息是独立的: 创建新记忆 (不标记 json_doc_id)
# 
# 在这个例子中:
# - "went to Tartine" 和 "Japan trip" 可能与 "bike ride" 相关
# - Trustcall 会决定是更新现有记忆还是创建新记忆

In [ ]:
# ================== 查看更新后的 Tool Calls ==================

# 打印所有 messages
# 预期看到两个 Memory tool calls:
# 1. 一个更新现有记忆 (包含所有相关信息)
# 2. 一个创建新记忆 (独立的新信息)
for m in result["messages"]:
    m.pretty_print()

# 输出分析:
# Tool Call 1:
#   content: 'Lance had a nice bike ride in San Francisco this morning. 
#             He went to Tartine and ate a croissant. 
#             He was thinking about his trip to Japan and going back this winter!'
#   - 这是更新操作,合并了多个相关信息
# 
# Tool Call 2:
#   content: 'Lance went to Tartine and ate a croissant. 
#             He was thinking about his trip to Japan and going back this winter!'
#   - 这是插入操作,创建新的独立记忆

# Trustcall 的并行操作:
# - 使用 parallel tool calling 同时进行更新和插入
# - 一次 LLM 调用完成多个操作,提高效率

In [ ]:
# ================== 查看更新后的 Responses ==================

# 打印所有 responses
# 显示更新后的记忆内容
for m in result["responses"]: 
    print(m)

# 输出:
# 1. Memory(content='Lance had a nice bike ride...')  - 更新后的记忆
# 2. Memory(content='Lance went to Tartine...')  - 新插入的记忆

# Collection 更新的结果:
# - 第一个记忆被扩充,包含了更多上下文信息
# - 第二个记忆是新创建的,包含独立的信息
# - 两个记忆都符合 Memory schema

这告诉我们,我们通过指定 `json_doc_id` 更新了 collection 中的第一个记忆。

In [ ]:
# ================== 查看 Metadata - 识别更新 vs 插入 ==================

# 打印所有 metadata
# 关键信息: json_doc_id 字段
for m in result["response_metadata"]: 
    print(m)

# 输出分析:
# 1. {'id': 'call_vxks...', 'json_doc_id': '0'}  - 有 json_doc_id
#    - 这表示更新操作
#    - json_doc_id='0' 对应 existing_memories 中的第一个记忆
#    - 使用这个 id 覆盖现有记忆
# 
# 2. {'id': 'call_Y4S3...'}  - 没有 json_doc_id
#    - 这表示插入操作
#    - 需要生成新的 UUID 作为 key

# json_doc_id 的作用:
# - 有 json_doc_id: 使用它作为 key,覆盖现有记忆
# - 无 json_doc_id: 生成新 UUID,插入新记忆
# 
# 这就是 Trustcall 实现同时更新和插入的机制!

# 在 chatbot 中的应用:
# - write_memory 函数会检查 json_doc_id
# - 如果存在,使用它作为 key 保存 (更新)
# - 如果不存在,生成新 UUID 作为 key 保存 (插入)

LangSmith trace: 

https://smith.langchain.com/public/ebc1cb01-f021-4794-80c0-c75d6ea90446/r

## 使用 collection schema 更新的 Chatbot

现在,让我们将 Trustcall 引入我们的 chatbot 来创建和更新记忆 collection。

In [ ]:
from IPython.display import Image, display

import uuid

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.store.memory import InMemoryStore
from langchain_core.messages import merge_message_runs
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.runnables.config import RunnableConfig
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.base import BaseStore

# ================== 初始化 Model ==================

model = ChatOpenAI(model="gpt-4o", temperature=0)

# ================== 定义 Memory Schema ==================

# Collection Schema: 每个记忆是独立的 Memory 对象
class Memory(BaseModel):
    """单个记忆条目 - Collection 中的基本单元"""
    content: str = Field(
        description="The main content of the memory. For example: User expressed interest in learning about French."
    )

# ================== 创建 Trustcall Extractor ==================

# Trustcall 知识点:
# - enable_inserts=True: 允许同时更新现有记忆和插入新记忆
# - 这是 Collection 管理的核心能力

trustcall_extractor = create_extractor(
    model,
    tools=[Memory],
    tool_choice="Memory",
    enable_inserts=True,  # 允许插入新记忆
)

# ================== 系统提示词 ==================

# Chatbot 主对话提示
MODEL_SYSTEM_MESSAGE = """You are a helpful chatbot. You are designed to be a companion to a user. 

You have a long term memory which keeps track of information you learn about the user over time.

Current Memory (may include updated memories from this conversation): 

{memory}"""

# Trustcall 提取提示
# 关键指令: "Use parallel tool calling to handle updates and insertions simultaneously"
# - 告诉 LLM 使用并行 tool calling
# - 同时处理更新和插入操作
TRUSTCALL_INSTRUCTION = """Reflect on following interaction. 

Use the provided tools to retain any necessary memories about the user. 

Use parallel tool calling to handle updates and insertions simultaneously:"""

# ================== call_model 节点 ==================

def call_model(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """
    主对话节点 - 从 store 加载记忆集合并生成个性化响应
    
    与 Profile Chatbot 的区别:
    - Profile: 加载单个 profile,格式化为结构化信息
    - Collection: 加载多个记忆,格式化为列表
    
    工作流程:
        1. 从 config 获取 user_id
        2. 从 store 搜索该用户的所有记忆
        3. 将记忆列表格式化为文本
        4. 注入系统提示并生成响应
    """
    
    # 获取 user_id
    user_id = config["configurable"]["user_id"]

    # ========== 从 Store 检索所有记忆 ==========
    # Collection 知识点:
    # - search(): 返回所有匹配 namespace 的 Item 对象
    # - 不是 get(),因为 Collection 有多个记忆
    namespace = ("memories", user_id)  # 注意: 使用 "memories"
    memories = store.search(namespace)

    # ========== 格式化记忆列表 ==========
    # Python 知识点:
    # - join(): 将列表元素连接为字符串
    # - f-string: 格式化每个记忆为 "- content"
    # - mem.value['content']: 从 Item 对象获取记忆内容
    
    # 格式化为带有项目符号的列表
    # 输出示例:
    # - User's name is Lance.
    # - User likes to bike around San Francisco.
    # - User enjoys going to bakeries.
    info = "\n".join(f"- {mem.value['content']}" for mem in memories)
    
    # 注入记忆到系统提示
    system_msg = MODEL_SYSTEM_MESSAGE.format(memory=info)

    # 生成响应
    response = model.invoke([SystemMessage(content=system_msg)]+state["messages"])

    return {"messages": response}

# ================== write_memory 节点 ==================

def write_memory(state: MessagesState, config: RunnableConfig, store: BaseStore):
    """
    记忆写入节点 - 使用 Trustcall 更新和插入记忆 collection
    
    核心功能:
    - 同时支持更新现有记忆和插入新记忆
    - 使用 json_doc_id 区分更新和插入
    - 高效管理记忆集合
    
    工作流程:
        1. 从 config 获取 user_id
        2. 从 store 检索现有记忆列表
        3. 准备 existing 参数 (key, tool_name, value 元组)
        4. 调用 Trustcall extractor 进行更新/插入
        5. 遍历结果,根据 json_doc_id 决定使用的 key
        6. 保存更新或新记忆到 store
    """
    
    # 获取 user_id
    user_id = config["configurable"]["user_id"]

    # 定义 namespace
    namespace = ("memories", user_id)

    # ========== 检索现有记忆 ==========
    existing_items = store.search(namespace)

    # ========== 准备 existing 参数 ==========
    # Trustcall Collection 更新的核心格式:
    # - List[Tuple[key, tool_name, value]]
    # - key: Store 中的 key (UUID 字符串)
    # - tool_name: "Memory"
    # - value: 记忆的字典表示
    
    tool_name = "Memory"
    existing_memories = (
        [(existing_item.key, tool_name, existing_item.value)  # 使用 Store 的 key
         for existing_item in existing_items]
        if existing_items
        else None
    )
    
    # 关键区别:
    # - Profile: existing = {"UserProfile": profile_dict}
    # - Collection: existing = [('uuid1', 'Memory', {...}), ('uuid2', 'Memory', {...})]

    # ========== 合并消息 ==========
    # LangChain 知识点:
    # - merge_message_runs(): 合并连续的相同角色消息
    # - 用于清理对话历史,避免重复
    updated_messages = list(merge_message_runs(
        messages=[SystemMessage(content=TRUSTCALL_INSTRUCTION)] + state["messages"]
    ))

    # ========== 调用 Trustcall Extractor ==========
    # 工作流程:
    # 1. Trustcall 分析对话和现有记忆
    # 2. 决定哪些记忆需要更新 (标记 json_doc_id)
    # 3. 决定哪些记忆需要插入 (不标记 json_doc_id)
    # 4. 生成相应的 tool calls
    # 5. 返回结果
    result = trustcall_extractor.invoke({
        "messages": updated_messages, 
        "existing": existing_memories
    })

    # ========== 保存记忆到 Store ==========
    # Python 知识点:
    # - zip(list1, list2): 并行遍历两个列表
    # - r: Memory 实例 (response)
    # - rmeta: 元数据字典 (response_metadata)
    
    for r, rmeta in zip(result["responses"], result["response_metadata"]):
        # ========== 决定使用的 Key ==========
        # Collection 更新的关键逻辑:
        # 
        # 1. 检查 rmeta 中是否有 json_doc_id:
        #    - 有: 这是更新操作,使用 json_doc_id 作为 key
        #    - 无: 这是插入操作,生成新 UUID 作为 key
        # 
        # 2. rmeta.get("json_doc_id", default):
        #    - 如果有 json_doc_id,返回它 (例如: 'uuid1')
        #    - 如果没有,返回 default (新生成的 UUID)
        
        # 保存到 store
        store.put(
            namespace,
            rmeta.get("json_doc_id", str(uuid.uuid4())),  # 更新用原key,插入用新key
            r.model_dump(mode="json"),  # 转换为字典
        )
    
    # Collection 保存的优势:
    # - 灵活性: 可以同时更新和插入
    # - 效率: 一次操作完成多个记忆的管理
    # - 智能: LLM 决定哪些记忆需要更新,哪些需要创建

# ================== 构建 Graph ==================

# 定义图 - 与 Profile Chatbot 相同的结构
builder = StateGraph(MessagesState)

# 添加节点
builder.add_node("call_model", call_model)
builder.add_node("write_memory", write_memory)

# 添加边 - 简单的线性流程
# START -> call_model -> write_memory -> END
builder.add_edge(START, "call_model")
builder.add_edge("call_model", "write_memory")
builder.add_edge("write_memory", END)

# ================== 初始化双记忆系统 ==================

# 长期记忆 (跨线程) - Store
# - 保存记忆 collection
# - 每个记忆是独立的条目
# - 使用 UUID 作为 key
across_thread_memory = InMemoryStore()

# 短期记忆 (线程内) - Checkpointer  
# - 保存对话历史
# - 每个线程独立
within_thread_memory = MemorySaver()

# ================== 编译 Graph ==================

# LangGraph 知识点:
# - checkpointer + store: 双记忆架构
# - 两者协同工作,提供完整的记忆能力
graph = builder.compile(
    checkpointer=within_thread_memory, 
    store=across_thread_memory
)

# 可视化图结构
display(Image(graph.get_graph(xray=1).draw_mermaid_png()))

# ================== 总结: Collection Chatbot vs Profile Chatbot ==================
#
# 1. Schema 设计:
#    - Profile: 固定结构,预定义字段 (name, location, interests)
#    - Collection: 灵活结构,每个记忆独立 (content)
#
# 2. 存储方式:
#    - Profile: 一个 key 保存整个 profile
#    - Collection: 多个 key,每个保存一个 Memory
#
# 3. 更新机制:
#    - Profile: JSON Patch 更新固定字段
#    - Collection: 同时支持更新现有记忆和插入新记忆
#
# 4. Trustcall 配置:
#    - Profile: 不需要 enable_inserts
#    - Collection: 需要 enable_inserts=True
#
# 5. 记忆格式化:
#    - Profile: 格式化为结构化信息 (Name: ..., Location: ...)
#    - Collection: 格式化为列表 (- memory1, - memory2, ...)
#
# 6. 使用场景:
#    - Profile: 适合结构化的用户信息 (个人档案)
#    - Collection: 适合开放式的记忆 (对话历史、知识点)
#
# 7. 扩展性:
#    - Profile: 需要预先定义所有字段
#    - Collection: 可以动态添加任意数量的记忆
#
# 8. 查询方式:
#    - Profile: get(namespace, key)
#    - Collection: search(namespace)

In [115]:
# We supply a thread ID for short-term (within-thread) memory
# We supply a user ID for long-term (across-thread) memory 
config = {"configurable": {"thread_id": "1", "user_id": "1"}}

# User input 
input_messages = [HumanMessage(content="Hi, my name is Lance")]

# Run the graph
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, my name is Lance
================================== Ai Message ==================================

Hi Lance! It's great to meet you. How can I assist you today?


In [116]:
# User input 
input_messages = [HumanMessage(content="I like to bike around San Francisco")]

# Run the graph
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

I like to bike around San Francisco
================================== Ai Message ==================================

That sounds like a lot of fun! San Francisco has some beautiful routes for biking. Do you have a favorite trail or area you like to explore?


In [117]:
# Namespace for the memory to save
user_id = "1"
namespace = ("memories", user_id)
memories = across_thread_memory.search(namespace)
for m in memories:
    print(m.dict())

{'value': {'content': "User's name is Lance."}, 'key': 'dee65880-dd7d-4184-8ca1-1f7400f7596b', 'namespace': ['memories', '1'], 'created_at': '2024-10-30T22:18:52.413283+00:00', 'updated_at': '2024-10-30T22:18:52.413284+00:00'}
{'value': {'content': 'User likes to bike around San Francisco.'}, 'key': '662195fc-8ea4-4f64-a6b6-6b86d9cb85c0', 'namespace': ['memories', '1'], 'created_at': '2024-10-30T22:18:56.597813+00:00', 'updated_at': '2024-10-30T22:18:56.597814+00:00'}


In [118]:
# User input 
input_messages = [HumanMessage(content="I also enjoy going to bakeries")]

# Run the graph
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

I also enjoy going to bakeries
================================== Ai Message ==================================

Biking and bakeries make a great combination! Do you have a favorite bakery in San Francisco, or are you on the hunt for new ones to try?


在新线程中继续对话。

In [119]:
# We supply a thread ID for short-term (within-thread) memory
# We supply a user ID for long-term (across-thread) memory 
config = {"configurable": {"thread_id": "2", "user_id": "1"}}

# User input 
input_messages = [HumanMessage(content="What bakeries do you recommend for me?")]

# Run the graph
for chunk in graph.stream({"messages": input_messages}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

What bakeries do you recommend for me?
================================== Ai Message ==================================

Since you enjoy biking around San Francisco, you might like to check out some of these bakeries that are both delicious and located in areas that are great for a bike ride:

1. **Tartine Bakery** - Located in the Mission District, it's famous for its bread and pastries. The area is vibrant and perfect for a leisurely ride.

2. **Arsicault Bakery** - Known for its incredible croissants, it's in the Richmond District, which offers a nice ride through Golden Gate Park.

3. **B. Patisserie** - Situated in Lower Pacific Heights, this bakery is renowned for its kouign-amann and other French pastries. The neighborhood is charming and bike-friendly.

4. **Mr. Holmes Bakehouse** - Famous for its cruffins, it's located in the Tenderloin, which is a bit more urban but still accessible by bike.

5.

### LangSmith 

https://smith.langchain.com/public/c87543ec-b426-4a82-a3ab-94d01c01d9f4/r

## Studio

![Screenshot 2024-10-30 at 11.29.25 AM.png](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/6732d0876d3daa19fef993ba_Screenshot%202024-11-11%20at%207.50.21%E2%80%AFPM.png)